In [ ]:
import os
import re
import json
import random
import pandas as pd

In [37]:
parent_dir = "explainer_data"
filename = "combined_chessexplainer.jsonl"
filepath = os.path.join(parent_dir, filename)

# Load data agnostic to parquet / json / jsonl
if filename.endswith('.parquet'):
    df = pd.read_parquet(filepath)
    data = df.to_dict('records')
    print(f"Loaded {len(data)} items from parquet file {filename}.")
elif filename.endswith('.json'):
    with open(filepath, 'r') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} items from JSON file {filename}.")
elif filename.endswith('.jsonl'):
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    print(f"Loaded {len(data)} items from JSONL file {filename}.")
else:
    raise ValueError(f"Unsupported file format: {filename}. Please use .parquet, .json, or .jsonl files.")

random.shuffle(data)
print(f"Shuffled {len(data)} items.")

Loaded 5350 items from JSONL file combined_chessexplainer.jsonl.
Shuffled 5350 items.


In [44]:
# Randomly sample / print data
sample = random.choice(data)
# print(sample)

for role, content in sample['chat']:
    print(f"Role: {role}\n{'-'*20}\n{content}\n\n")

Role: system
--------------------
chess_task_sysprompt.txt


Role: user
--------------------
Here is a board in a game you're currently playing. I want you to think through some possible moves you could make and how those moves will likely play out. You may find it helpful to roll-out each line assuming your opponent plays near-optimally. You may also find it helpful to consider the value of the final board state after each roll-out.

After you think through your various moves, please end by telling me your chosen move (in UCI notation) within answer tags.

8| r . . . . . . .
7| p p k . q . p p
6| . . . . . . . .
5| P . n Q . . . .
4| . . B . . . . .
3| . . . . . . P b
2| . . n . . P . P
1| R . . . . . K .
   _ _ _ _ _ _ _ _
   A B C D E F G H

- It is White’s turn to move.
- No castling rights available.
- No en passant target square.
- Halfmove clock: 2
- Fullmove number: 25


Role: assistant
--------------------
Interesting, let's consider some moves here.
<think>
Ok, what if we pla

## Code to clean up / concatenate existing data

In [35]:
# Example usage:
parent_dir = "explainer_data"
filenames = [
    "explainer_clean_100_1558_15.parquet",
    "explainer_clean_1250.parquet",
    "explanations_0_1000_0104_16.parquet",
    "explanations_1_1000_0330_16.parquet",
    "explanations_2_1000_0557_16.parquet",
    "explanations_3_1000_0826_16.parquet"
]
filepaths = [os.path.join(parent_dir, fn) for fn in filenames]
output_file = "combined_chessexplainer.jsonl"
system_prompt_file = "chess_task_sysprompt.txt"  # Or "llama4_default_sysprompt.txt"


HEADER_PATTERN = re.compile(
    r"<\|start_header_id\|>(\w+)<\|end_header_id\|>\n?(.*?)(?=(<\|start_header_id\|>|\Z|<\|eot_id\|>))",
    re.DOTALL
)

def extract_dialogue(sample):
    text = sample['prompt']
    result = []
    for match in HEADER_PATTERN.finditer(text):
        role, content = match.group(1), match.group(2).strip()
        # Remove Llama tags and both eot_id types
        content = re.sub(r"<\|.*?\|>|<eot_id>", "", content).strip()
        if role == "system":
            content = "chess_task_sysprompt.txt"
        if role == "user":
            prefix = "Here is a board in a game you're currently playing. I want you to think through some possible moves you could make and how those moves will likely play out. You may find it helpful to roll-out each line assuming your opponent plays near-optimally. You may also find it helpful to consider the value of the final board state after each roll-out.\n\nAfter you think through your various moves, please end by telling me your chosen move (in UCI notation) within answer tags.\n\n"
            content = prefix + content
        if content:
            result.append((role, content))
    completion = sample.get('completion', '').strip()
    completion = re.sub(r'(<\|eot_id\|>|<eot_id>)\s*$', '', completion).strip()
    if completion:
        result.append(('assistant', completion))
    return result

def convert_and_save(parquet_paths, output_path):
    all_dialogues = []
    for path in parquet_paths:
        df = pd.read_parquet(path)
        all_dialogues.extend(
            {"chat": extract_dialogue(row)} for row in df.to_dict('records')
        )
    # Save as JSONL
    with open(output_path, 'w', encoding='utf-8') as f:
        for d in all_dialogues:
            json.dump(d, f, ensure_ascii=False)
            f.write('\n')
    print(f"Saved {len(all_dialogues)} dialogues to {output_path}")


output_path = os.path.join(parent_dir, output_file)
convert_and_save(filepaths, output_path)

Saved 5350 dialogues to explainer_data\combined_chessexplainer.jsonl
